In [ ]:
from sashimi.hardware.scanning.ni import *
from nidaqmx.task import Task
from sashimi.config import read_config
from sashimi.waveforms import SawtoothWaveform, TriangleWaveform

import time
import matplotlib.pyplot as plt
import numpy as np

conf = read_config()

In [ ]:
# Generate a Trigger  
#z galvos and xy galvos

write_task_z= Task()
write_task_z.ao_channels.add_ao_voltage_chan("Dev3/ao0:3")

write_task_xy= Task()
write_task_xy.ao_channels.add_ao_voltage_chan("Dev2/ao0:1")

# Galvos

In [ ]:
#init variables

n_samples = 1000  # number of samples to write 
sample_rate= 400  # sample rate

#min max waveform
lateral_waveform_min = 0 
lateral_waveform_max = 2

#frequency
frequency_xy = 5.
frequency_z =4.

waveform_xy = TriangleWaveform(
            frequency=frequency_xy,
            vmin=lateral_waveform_min,
            vmax=lateral_waveform_max,
        )
waveform_z = TriangleWaveform(
            frequency=frequency_z,
            vmin=lateral_waveform_min,
            vmax=lateral_waveform_max,
        )

In [ ]:
# init arrays where to write inputs
xy_array = np.zeros((2, n_samples))
z_array = np.zeros((4, n_samples))

In [ ]:
# init time array
t = np.arange(n_samples) / sample_rate
shifted_time = t.copy()

In [ ]:
i_sample = 0

shifted_time[:] = t + i_sample / sample_rate

In [ ]:
#fill arrays with initial values

z_array[1, :] = waveform_z.values(shifted_time)
z_array[2, :] = waveform_z.values(shifted_time)
xy_array[0, :] = waveform_xy.values(shifted_time)
xy_array[1, :] = waveform_xy.values(shifted_time)

In [ ]:
#sample loop to move the galvos in z

t_start = time.time()
i=0
while time.time() - t_start < 7:  #loops for 7 sec
    
    i_sample +=1  #next sample

    shifted_time[:] = t + i_sample / sample_rate  #update time array
    
    z_array[1, :] = waveform_z.values(shifted_time)  #update waveform array
    z_array[2, :] = waveform_z.values(shifted_time)

    write_task_z.write(z_array, auto_start=True)  #write waveform array


In [ ]:
#sample loop to move the galvos in xy

t_start = time.time()
i = 0
while time.time() - t_start < 7:  #loops for 7 sec
    
    i_sample +=1  #next sample

    shifted_time[:] = t + i_sample / sample_rate   #update time array
    
    xy_array[0, :] = waveform_xy.values(shifted_time)  #update waveform array
    xy_array[1, :] = waveform_xy.values(shifted_time)

    write_task_xy.write(xy_array, auto_start=True)  #write waveform array

# Piezo


In [ ]:
#init variables

n_samples = 1000  # number of samples to write 
sample_rate= 400  # sample rate

#min max piezo
piezo_min = 30
piezo_max = 300

#frequency
frequency_piezo = 2.



waveform_z = SawtoothWaveform(
            frequency=frequency_piezo,
            vmin=piezo_min,
            vmax=piezo_max,
        )

In [ ]:
#init time array and array where to write input

t = np.arange(n_samples) / sample_rate
shifted_time = t.copy()
z_array = np.zeros((4, n_samples))
scale =0.025 # scale factor from input to actual value to write

In [ ]:
#sample loop to move the piezo
t_start = time.time()
i=0
while time.time() - t_start < 10:
    
    i_sample +=1   #next sample

    shifted_time[:] = t + i_sample / sample_rate  #update time array
    
    z_array[0, :] = waveform_z.values(shifted_time) * scale  #update waveform array

    write_task_z.write(z_array, auto_start=True)  #write waveform array